Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [16]:
#logistics model with SGD + L2 
def run_logistics_sgd_l2(batch_size, beta, num_steps):
    graph_logistics_sgd_l2 = tf.Graph()
    with graph_logistics_sgd_l2.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.nn.l2_loss(weights) * beta

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


    # run it

    with tf.Session(graph=graph_logistics_sgd_l2) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


In [17]:
batch_size = 128
beta = 0.01
num_steps = 3001

run_logistics_sgd_l2(batch_size, beta, num_steps)

Initialized
Minibatch loss at step 0: 44.298759
Minibatch accuracy: 14.8%
Validation accuracy: 13.8%
Minibatch loss at step 500: 0.727485
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.800813
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.566130
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.648398
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.782503
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.784393
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Test accuracy: 88.6%


In [43]:
#neural network with SGD + L2 + dropout

def run_nn_sgd_l2(batch_size, beta, num_steps):
    run_nn_sgd_l2_dropout(batch_size, beta, num_steps, False)

def run_nn_sgd_l2_dropout(batch_size, beta, num_steps, is_dropout=True):
    graph_nn = tf.Graph()
    with graph_nn.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      num_hidden_nodes = 1024

      # Variables.
      weights_i = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
      biases_i = tf.Variable(tf.zeros([num_hidden_nodes]))

      weights_o = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
      biases_o = tf.Variable(tf.zeros([num_labels]))

      def forward_prop(ds, w_i, b_i, w_o, b_o):
        return tf.matmul(
          tf.nn.relu(
              tf.matmul(ds, w_i) + b_i
          ), w_o) + b_o
    
      def forward_prop_dropout(ds, w_i, b_i, w_o, b_o):
        return tf.matmul(
          tf.nn.dropout(
              tf.nn.relu(tf.matmul(ds, w_i) + b_i), 0.5
          ), w_o
        ) + b_o
    
      # Training computation.
      if is_dropout:
        logits = forward_prop_dropout(tf_train_dataset, weights_i, biases_i, weights_o, biases_o)
      else:
        logits = forward_prop(tf_train_dataset, weights_i, biases_i, weights_o, biases_o)

      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (beta * (tf.nn.l2_loss(weights_i) + tf.nn.l2_loss(weights_o)))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights_i, biases_i, weights_o, biases_o))
      test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights_i, biases_i, weights_o, biases_o))


    #run it
    with tf.Session(graph=graph_nn) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


In [46]:
batch_size = 128
beta = 0.001
num_steps = 3001

run_nn_sgd_l2(batch_size, beta, num_steps)

Initialized
Minibatch loss at step 0: 615.358154
Minibatch accuracy: 14.8%
Validation accuracy: 29.1%
Minibatch loss at step 500: 198.677811
Minibatch accuracy: 84.4%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 114.323944
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 68.371544
Minibatch accuracy: 89.1%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 41.173523
Minibatch accuracy: 92.2%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 25.163031
Minibatch accuracy: 91.4%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.476767
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [48]:
# with just few batches
batch_size = 128
beta = 0.001
num_steps = 10

In [49]:
print('With L2:')
run_logistics_sgd_l2(batch_size, beta, num_steps)

print('\n\nWithout L2:')
run_logistics_sgd_l2(batch_size, 0, num_steps)

With L2:
Initialized
Minibatch loss at step 0: 633.617554
Minibatch accuracy: 13.3%
Validation accuracy: 26.5%
Test accuracy: 77.0%


Without L2:
Initialized
Minibatch loss at step 0: 342.237793
Minibatch accuracy: 7.0%
Validation accuracy: 28.4%
Test accuracy: 80.5%


In [50]:
print('With L2:')
run_nn_sgd_l2(batch_size, beta, num_steps)

print('\n\nWithout L2:')
run_nn_sgd_l2(batch_size, 0, num_steps)

With L2:
Initialized
Minibatch loss at step 0: 682.503235
Minibatch accuracy: 8.6%
Validation accuracy: 25.2%
Test accuracy: 80.1%


Without L2:
Initialized
Minibatch loss at step 0: 424.563721
Minibatch accuracy: 10.2%
Validation accuracy: 28.6%
Test accuracy: 74.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [45]:
batch_size = 128
beta = 0.001
num_steps = 3001

run_nn_sgd_l2_dropout(batch_size, beta, num_steps)

Initialized
Minibatch loss at step 0: 775.724243
Minibatch accuracy: 9.4%
Validation accuracy: 30.2%
Minibatch loss at step 500: 210.526810
Minibatch accuracy: 75.8%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 119.015320
Minibatch accuracy: 74.2%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 70.021873
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 41.560383
Minibatch accuracy: 84.4%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 25.321037
Minibatch accuracy: 82.0%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 15.632507
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Test accuracy: 91.8%


In [47]:
batch_size = 128
beta = 0.001
num_steps = 10

run_nn_sgd_l2_dropout(batch_size, beta, num_steps)

Initialized
Minibatch loss at step 0: 807.790161
Minibatch accuracy: 14.1%
Validation accuracy: 30.1%
Test accuracy: 80.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [195]:
def run_decay_nn_sgd_l2_dropout(batch_size, beta, num_steps, deacy_steps, decay_rate, is_dropout=True):
    graph_nn = tf.Graph()
    with graph_nn.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      num_hidden_nodes = 1024

      # Variables.
      weights_i = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
      biases_i = tf.Variable(tf.zeros([num_hidden_nodes]))

      weights_o = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
      biases_o = tf.Variable(tf.zeros([num_labels]))

      def forward_prop(ds, w_i, b_i, w_o, b_o):
        return tf.matmul(
          tf.nn.relu(
              tf.matmul(ds, w_i) + b_i
          ), w_o) + b_o
    
      def forward_prop_dropout(ds, w_i, b_i, w_o, b_o):
        return tf.matmul(
          tf.nn.dropout(
              tf.nn.relu(tf.matmul(ds, w_i) + b_i), 0.5
          ), w_o
        ) + b_o
    
      # Training computation.
      if is_dropout:
        logits = forward_prop_dropout(tf_train_dataset, weights_i, biases_i, weights_o, biases_o)
      else:
        logits = forward_prop(tf_train_dataset, weights_i, biases_i, weights_o, biases_o)

      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (beta * (tf.nn.l2_loss(weights_i) + tf.nn.l2_loss(weights_o)) / 2)

      # Optimizer.
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, deacy_steps, decay_rate)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights_i, biases_i, weights_o, biases_o))
      test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights_i, biases_i, weights_o, biases_o))


    #run it
    with tf.Session(graph=graph_nn) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


In [196]:
batch_size = 128
beta = 0.001
num_steps = 3001 * 2
decay_step=500
decay_rate=0.95

run_decay_nn_sgd_l2_dropout(batch_size, beta, num_steps, decay_step, decay_rate)

Initialized
Minibatch loss at step 0: 631.445312
Minibatch accuracy: 6.2%
Validation accuracy: 28.0%
Minibatch loss at step 500: 164.186340
Minibatch accuracy: 73.4%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 102.542458
Minibatch accuracy: 71.1%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 81.062073
Minibatch accuracy: 75.8%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 62.245358
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 51.260117
Minibatch accuracy: 79.7%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 42.208858
Minibatch accuracy: 78.9%
Validation accuracy: 83.2%
Minibatch loss at step 3500: 35.435528
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 29.841913
Minibatch accuracy: 85.9%
Validation accuracy: 84.1%
Minibatch loss at step 4500: 25.390028
Minibatch accuracy: 82.8%
Validation accuracy: 84.4%
Minibatch loss at step 5000: 21.574705
Minibatch accuracy: 82.8%
Valid

In [63]:
batch_size = 128
beta = 0.001
num_steps = 3001 * 2
decay_step=500
decay_rate=0.98

run_decay_nn_sgd_l2_dropout(batch_size, beta, num_steps, decay_step, decay_rate)

Initialized
Minibatch loss at step 0: 821.997986
Minibatch accuracy: 7.8%
Validation accuracy: 27.5%
Minibatch loss at step 500: 216.470398
Minibatch accuracy: 73.4%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 120.009338
Minibatch accuracy: 78.1%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 72.708885
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 45.291584
Minibatch accuracy: 84.4%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 28.534210
Minibatch accuracy: 78.9%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 18.431175
Minibatch accuracy: 83.6%
Validation accuracy: 85.0%
Minibatch loss at step 3500: 12.047112
Minibatch accuracy: 84.4%
Validation accuracy: 85.9%
Minibatch loss at step 4000: 8.007166
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 4500: 5.394007
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 5000: 3.791618
Minibatch accuracy: 85.9%
Validati

In [240]:
#multi layer nn
def run_decay_mlayer_nn_sgd_l2(batch_size, beta, num_steps, lr_initial, lr_deacy_steps, lr_decay_rate, hidden_layers, is_dropout=True):
    graph_nn = tf.Graph()
    with graph_nn.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                            shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        
        if type(hidden_layers) is not list:
            hidden_layers = []
        # Variables.
        layers = [image_size * image_size] + hidden_layers + [num_labels]
        weights = []
        biases = []
        
        for idx, l in enumerate(layers):
            if idx == len(layers) -1:
                break
            else:
                weights.append(tf.Variable(tf.truncated_normal([layers[idx], layers[idx + 1]])))
                biases.append(tf.Variable(tf.zeros([layers[idx + 1]])))

        def forward_prop(input, weights, biases, apply_dropout=False):
            if len(weights) != 1:
                _logits = tf.nn.relu(tf.matmul(input, weights[0]) + biases[0])
                if apply_dropout:
                    _logits = tf.nn.dropout(_logits, 0.5)
                return forward_prop(_logits, weights[1:], biases[1:], apply_dropout)
            else:
                return tf.matmul(input, weights[0]) + biases[0]
            
        
        def get_regularization(beta, weights):
            losses = [tf.nn.l2_loss(w) for w in weights]
            return tf.add_n(losses) / len(weights) * beta
                
        
        # Training computation.
        logits = forward_prop(tf_train_dataset, weights, biases, is_dropout)
        
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=tf_train_labels, logits=logits)) + get_regularization(beta, weights)

        # Optimizer.
        global_step = tf.Variable(0)  # count the number of steps taken.
        learning_rate = tf.train.exponential_decay(lr_initial, global_step, lr_deacy_steps, lr_decay_rate)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights, biases))
        test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights, biases))
        
    #run it
    with tf.Session(graph=graph_nn) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
            
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
                
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


In [244]:
hidden_layers = [1024]
batch_size = 128
beta = 0.001
num_steps = 3001 * 3
lr_initial=0.5
lr_decay_step=500
lr_decay_rate=0.98
batch_size = 128
run_decay_mlayer_nn_sgd_l2(batch_size, beta, num_steps, lr_initial, lr_decay_step, lr_decay_rate, hidden_layers, True)

Initialized
Minibatch loss at step 0: 671.171753
Minibatch accuracy: 7.8%
Validation accuracy: 31.6%
Minibatch loss at step 500: 158.457306
Minibatch accuracy: 72.7%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 104.380592
Minibatch accuracy: 74.2%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 78.187340
Minibatch accuracy: 84.4%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 58.937981
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 46.977081
Minibatch accuracy: 79.7%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 37.287144
Minibatch accuracy: 78.9%
Validation accuracy: 83.2%
Minibatch loss at step 3500: 30.061758
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 4000: 24.271507
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%
Minibatch loss at step 4500: 19.408957
Minibatch accuracy: 85.2%
Validation accuracy: 84.5%
Minibatch loss at step 5000: 15.972607
Minibatch accuracy: 83.6%
Valid

In [243]:
hidden_layers = [1024]
batch_size = 128
beta = 0.5
num_steps = 10000
lr_initial=0.5
lr_decay_step=1000
lr_decay_rate=0.9
batch_size = 128
run_decay_mlayer_nn_sgd_l2(batch_size, beta, num_steps, lr_initial, lr_decay_step, lr_decay_rate, hidden_layers, True)

Initialized
Minibatch loss at step 0: 79137.296875
Minibatch accuracy: 13.3%
Validation accuracy: 38.5%
Minibatch loss at step 500: 1.933796
Minibatch accuracy: 60.2%
Validation accuracy: 60.4%
Minibatch loss at step 1000: 1.942253
Minibatch accuracy: 62.5%
Validation accuracy: 68.3%
Minibatch loss at step 1500: 1.845820
Minibatch accuracy: 65.6%
Validation accuracy: 66.4%
Minibatch loss at step 2000: 1.908276
Minibatch accuracy: 67.2%
Validation accuracy: 62.6%
Minibatch loss at step 2500: 1.961326
Minibatch accuracy: 66.4%
Validation accuracy: 67.4%
Minibatch loss at step 3000: 1.985622
Minibatch accuracy: 64.1%
Validation accuracy: 67.8%
Minibatch loss at step 3500: 1.968703
Minibatch accuracy: 68.0%
Validation accuracy: 64.1%
Minibatch loss at step 4000: 1.940827
Minibatch accuracy: 58.6%
Validation accuracy: 74.0%
Minibatch loss at step 4500: 1.886110
Minibatch accuracy: 65.6%
Validation accuracy: 64.7%
Minibatch loss at step 5000: 1.827527
Minibatch accuracy: 75.8%
Validation acc